# 原始积累
> 分页：通过底部tabs实现
News页面：底部tab触发news_tab_fragment,其为黄鹤楼的tabactivity，其中又含有tabs,此tab触发真正的新闻目录，即黄鹤楼的tabfragemnt（注意，四个目录tab关联四种不同的fragment）

> 增删tab用李方法

> 新闻列表用暑培方法

> Fragment传参/调用见csdn

> 烟雨莽苍苍，龟蛇锁大江。

> 传参：
传参之一：四大主界面之间通过Mainactivity作为中介进行传参，其中主界面传入MainActivity通过interface+OnAttach实现，Mainactivity传入主界面通过Bundle实现；
传参之二：各个主界面与其所调用的fragment之间通过Bundle传参。
“使用接口在 Fragment 之间传递参数更加灵活且适用于各种场景，而使用 Bundle 则更适合在 Fragment 跳转时需要传递数据的情况。”

>  API1：根据索引返回符合category或搜索条件的ArrayList<News>
API2:  根据数据库返回历史/收藏的ArrayList<News>

>列表界面：将这些返回的list显示在recyclelist中，并提供点击进入详情页面服务（传参+fragment调用）
详情页面：根据传入的参数(很可能是News的一个实例)显示新闻的所用内容


- 主线程报错
- 将获取NewsList放在另一个线程，并在主线程中join等待它执行完（否则无法正确传参，可能出现get到的参数是初始化时候的newslist）
#
- 如何解决recyclerview各个条目图片错乱问题？
- 回收机制，笨办法是在Adapter里重载etItemViewType，但据说会降低性能，待优化
#
- image链接不合法，Glide加载图片失败
- Glide设置addListener，onResourceReady里设置图片变成visible
#
- http不支持加载
- add android:usesCleartextTraffic=“true” to application

# 致命性TODO（已解决）
- glide 磁盘缓存大小之自定义，千万别装不下了 ：一切由glide自行处理，不需离线图片一切不是问题
- 线程安全：Sol：java自带的线程锁机制，synchronized
- search 界面布局问题！！  <mark>视频离线问题！！</mark> 日期输入格式说明(英文；两位) 
- 数据库读写权限：无！数据库销毁:Ok ;数据库是否需要开线程 :小量不需要（ANR需要5秒，字符串数据库应该来得及）
#
### 上拉下拉
- SmartRefreshLayout：依赖库问题
-  implementation 'com.scwang.smartrefresh:SmartRefreshLayout:1.1.3'等等根本不对，必须导入"io.github.scwang90:refresh-layout-kernel:2.0.5"才可以
- 之前以为是gradle用kotlin/groovy导致的差异/引进库时加载中断导致的库不全，徒增焦虑————没良心的CSDN，发的blog竟连依赖库都不对！
- 仍不对
- 最终解决：在gradle.properties加入```android.enableJetifier=true```，原理是：原始项目不支持AndroidX，必须加这句将之转换到AndroidX
- 核心教训：<mark>用第三方库出事时(尤其是配置出事时),解决方法不是查找相关社区/问AI,而是找到官网读取详细说明</mark>
#
### 网络请求与线程机制
- 问题：主线程不许使用网络请求
- 策略：开辟子线程
- 问题：主线程必须等到子线程结束后才获得子线程数据，以更新recyclerview
- 策略：主线程Thread.join()
- 问题: 在oncreateview里主线程等待子线程结束会特别慢&引发ANR错误
- 策略：在子线程里用requireActivity().runOnUiThread切换到主线程
- 问题：newsArrayList = list;后，直接调用notifyDataSetChanged();无反应
- 策略：问题在于初始化adapter后在外部重新赋值arraylist（注意，不是说.add之类基于原有指针的操作，而是赋值号"=",实为改变外部list的指针指向）时不能传递给内部,解决方案是设置接口显示传参改变内部arraylist
- 问题：requireActivity().runOnUiThread会导致在activity为关联到fragment时出现异常IllegalStateException
- 策略： 改用new Handler(Looper.getMainLooper()).post
- 上课上讲的很关键！
#
- .fitCenter()//这句特别重要，否则图片不完整！
### 回退后恢复原始状态/避免重新下载(杜一阳学长的TODO)/列表中看过的离线加载
- 安卓自带的回退键即可
- 安卓手机自带的回退键是通过系统提供的onBackPressed()方法实现的。该方法在Activity中被调用，用于处理用户按下“回退”键的事件。
当用户按下“回退”键时，系统会先判断当前Activity是否有处理返回事件的特定逻辑，如果存在，则执行该逻辑；否则，系统会回调onBackPressed()方法，以执行默认的返回逻辑，如关闭当前Activity并返回上一个Activity或桌面。如果方法A是在onCreateView()中实现的，那么当Fragment从后台恢复到前台时，不会重新调用onCreateView()方法。因此，方法A的效果会保持。
当Fragment被覆盖或从回退栈中弹出并恢复到前台时，它的视图层次结构（包括通过onCreateView()方法创建的视图）将得到保留，除非显式地销毁该Fragment。
所以，如果你在onCreateView()方法中执行了某些操作，例如设置UI控件的属性、绑定监听器等，当Fragment从后台恢复到前台时，这些设置将保持不变，并且不需要重复执行。
需要注意的是，当Fragment从后台恢复到前台时，系统将调用onStart()和onResume()方法，你可以在这些方法中执行其他需要更新的操作，如数据刷新、网络请求等。
- 离线加载图片 ```RequestOptions requestOptions = new RequestOptions().diskCacheStrategy(DiskCacheStrategy.ALL);
Glide.with(context)
    .load(imageUrl)
    .apply(requestOptions)
    .into(imageView);```
# 感想
- 查看日志/报错信息非常关键！
#
- 典型的误区思维：尝试实验--若成功则不深究原理，若失败则找问题-->解决问题
- 莫名所以地失败；莫名所以地解决
# 
- 提出需求-->问ChatGPT-->响应代码能通则OK,否则循环直到能通(针对回答的正确性问题)